<a href="https://colab.research.google.com/github/marcellamj/soulcode-martech/blob/main/AD2_Fundamentos_de_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Fundamentos de Pyspark**

Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

Período: Semana 7

Professor: Douglas Ribeiro


## **Importações**

* Instalar no computador do Google
* Importar ferramentas do ambiente clusterizado
* Criar sessão Spark em um cluster e deixar visualização das tabelas mais amigável

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)                         # Deixar a visualição das tabelas mais amigável
spark

In [ ]:
import pandas as pd

## **Dataframe**

### Criar usando Python

In [ ]:
# Criando um dataframe a partir de listas Python
nome = ['Douglas','Daniela','Pedro','Maria','Eduardo','Ester']
idade = [45,7,65,64,37,37]
altura = [1.85,1.23,1.75,1.67,1.82,1.73]
peso = [70,22,87,64,96,68]
df_pd = pd.DataFrame(zip(nome,idade,altura,peso),columns=['nome','idade','altura','peso'])
display(df_pd)
df_pd.info()

,nome,idade,altura,peso
0,Douglas,45,1.85,70
1,Daniela,7,1.23,22
2,Pedro,65,1.75,87
3,Maria,64,1.67,64
4,Eduardo,37,1.82,96
5,Ester,37,1.73,68


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nome    6 non-null      object 
 1   idade   6 non-null      int64  
 2   altura  6 non-null      float64
 3   peso    6 non-null      int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 320.0+ bytes


### Criar usando PySpark

* Importando estrutura da tabela (structfield)
* Criação do Schema, esqueleto da tabela
* Criação do objeto, nível Python, que contém as tuplas com os registros que farão parte da tabela
* Criação do dataframe py spark, juntando a estrutura da tabela com o objeto que contém os registros
* Imprimir o schema do df_py
* Mostrar o df_py

In [ ]:
# A função StructType é usada para definir a estrutura de uma tabela
# A função StructField é usada para definir os atributos da tabela
# (String, Integer -inteiro-, Double -float-)Type define o tipo da variável
# True significa que aceita valor nulo

from pyspark.sql.types import *
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
schema = StructType([ \
    StructField('nome',StringType(),True), \
    StructField('idade',IntegerType(),True), \
    StructField('altura',DoubleType(),True), \
    StructField('peso', DoubleType(), True), \
  ])
dados = [('Douglas',45,1.85,70.),
    ('Daniela',7,1.23,22.),
    ('Pedro',65,1.75,87.),
    ('Maria',64,1.67,64.),
    ('Eduardo',37,1.82,96.),
    ('Ester',37,1.73,68.)
  ]

df_py = spark.createDataFrame(data=dados,schema=schema)
df_py.printSchema()
df_py.show()

root
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- altura: double (nullable = true)
 |-- peso: double (nullable = true)

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|Daniela|    7|  1.23|22.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
|Eduardo|   37|  1.82|96.0|
|  Ester|   37|  1.73|68.0|
+-------+-----+------+----+



### Criar df PySpark a partir de um df Pandas

In [ ]:
df_py2 = spark.createDataFrame(df_pd)
df_py2.printSchema()
df_py2.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- nome: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- altura: double (nullable = true)
 |-- peso: long (nullable = true)

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|  70|
|Daniela|    7|  1.23|  22|
|  Pedro|   65|  1.75|  87|
|  Maria|   64|  1.67|  64|
|Eduardo|   37|  1.82|  96|
|  Ester|   37|  1.73|  68|
+-------+-----+------+----+



### Filter

In [ ]:
# Filter no Pandas
display(df_pd[0:1])

,nome,idade,altura,peso
0,Douglas,45,1.85,70


In [ ]:
# Filter no Pyspark
df_py.filter(df_py.nome == 'Douglas')

nome,idade,altura,peso
Douglas,45,1.85,70.0


In [ ]:
# Filter no Pypark (alternativa estilo sql)
df_py.where(df_py.nome == 'Douglas')

nome,idade,altura,peso
Douglas,45,1.85,70.0


### Merge

Unir 2 dataframes

In [ ]:
# Pandas
nome2 = ['Douglas','Daniela']
idade2 = [45,7]
altura2 = [1.85,1.23]
peso2 = [70,22]
df_pd2 = pd.DataFrame(zip(nome2,idade2,altura2,peso2),columns=['nome','idade','altura','peso'])
display(df_pd2)

nome3 = ['Pedro','Maria']
idade3 = [69,68]
altura3 = [1.65,1.67]
peso3 = [96,65]
df_pd3 = pd.DataFrame(zip(nome3,idade3,altura3,peso3),columns=['nome','idade','altura','peso'])
display(df_pd3)

,nome,idade,altura,peso
0,Douglas,45,1.85,70
1,Daniela,7,1.23,22


,nome,idade,altura,peso
0,Pedro,69,1.65,96
1,Maria,68,1.67,65


In [ ]:
# Merge com Pandas (concat)
casa=pd.concat([df_pd2,df_pd3], ignore_index=True)
display(casa)

,nome,idade,altura,peso
0,Douglas,45,1.85,70
1,Daniela,7,1.23,22
2,Pedro,69,1.65,96
3,Maria,68,1.67,65


In [ ]:
# Unir dois dataframes no PySpark -linha por linha
douglas = df_py.filter(df_py.nome == 'Douglas')
daniela = df_py.filter(df_py.nome == 'Daniela')
filtro = douglas.union(daniela)
filtro.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|Daniela|    7|  1.23|22.0|
+-------+-----+------+----+



In [ ]:
# Fazer linha por linha
n1 = df_py.filter(df_py.nome == 'Pedro')
n2 = df_py.filter(df_py.nome == 'Maria')
n3 = n1.union(n2)
n3.show()

+-----+-----+------+----+
| nome|idade|altura|peso|
+-----+-----+------+----+
|Pedro|   65|  1.75|87.0|
|Maria|   64|  1.67|64.0|
+-----+-----+------+----+



In [ ]:
df = filtro.union(n3)
df.show()

+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|70.0|
|Daniela|    7|  1.23|22.0|
|  Pedro|   65|  1.75|87.0|
|  Maria|   64|  1.67|64.0|
+-------+-----+------+----+



In [ ]:
# Transformar pd para spark e unir os dois dataframes
df_py3 = spark.createDataFrame(df_pd2)
df_py4 = spark.createDataFrame(df_pd3)
df_py5 = df_py2.union(df_py3)
df_py5.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-------+-----+------+----+
|   nome|idade|altura|peso|
+-------+-----+------+----+
|Douglas|   45|  1.85|  70|
|Daniela|    7|  1.23|  22|
|  Pedro|   65|  1.75|  87|
|  Maria|   64|  1.67|  64|
|Eduardo|   37|  1.82|  96|
|  Ester|   37|  1.73|  68|
|Douglas|   45|  1.85|  70|
|Daniela|    7|  1.23|  22|
+-------+-----+------+----+



### Acessar dados de linha

In [ ]:
# Acessando os dados de uma linha do Pandas
df_pd.iloc[0]

nome      Douglas
idade          45
altura       1.85
peso           70
Name: 0, dtype: object

In [ ]:
# Guardando os dados de uma linha do Pandas
tio=df_pd.iloc[0]
print(tio)

nome      Douglas
idade          45
altura       1.85
peso           70
Name: 0, dtype: object


In [ ]:
# Acessando os dados de uma linha do PySpark
df_py.collect()[0]

Row(nome='Douglas', idade=45, altura=1.85, peso=70.0)

In [ ]:
# Guardando os dados de uma linha do PySpark
tio_py = df_py.collect()[0]
display(tio_py)

Row(nome='Douglas', idade=45, altura=1.85, peso=70.0)

### Acessar dados de linha e coluna

In [ ]:
# Acessando um dado específico de uma [linha][coluna] do Pandas
df_pd.iloc[0][0]

'Douglas'

In [ ]:
# Acessando um intervalo de [colunas][intervalo linhas] do Pandas
df_pd[['nome','peso']].iloc[1:4][0:2]

,nome,peso
1,Daniela,22
2,Pedro,87


In [ ]:
# Guardando os dados pelo endereço
tio_py2 = df_py.collect()[0][3]
print(tio_py2)

70.0


In [ ]:
# Guardando um valor específico do Pyspark pelo endereço
tio_py2 = df_py.collect()[0][3]
type(tio_py2)

float

In [ ]:
# Acessando os dados de uma linha do PySpark
df_py.collect()[0][0]

'Douglas'

Inserindo uma nova coluna em um dataframe pandas por atribuição direta

In [ ]:
# Inserir uma nova coluna em um df pelo Pandas
df_pd['sexo'] = ['M','F','M','F','M','F']
display(df_pd)

,nome,idade,altura,peso,sexo
0,Douglas,45,1.85,70,M
1,Daniela,7,1.23,22,F
2,Pedro,65,1.75,87,M
3,Maria,64,1.67,64,F
4,Eduardo,37,1.82,96,M
5,Ester,37,1.73,68,F


In [ ]:
# Inserindo uma nova coluna no PySpark via mudança de schema
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
dados = [('Douglas',45,1.85,70.,'M'),
    ('Daniela',7,1.23,22.,'F'),
    ('Pedro',65,1.75,87.,'M'),
    ('Maria',64,1.67,64.,'F'),
    ('Eduardo',37,1.82,96.,'M'),
    ('Ester',37,1.73,68.,'F')
  ]

schema = StructType([ \
    StructField('nome',StringType(),True), \
    StructField('idade',IntegerType(),True), \
    StructField('altura',DoubleType(),True), \
    StructField('peso', DoubleType(), True), \
    StructField('sexo', StringType(), True), \
  ])
df_py = spark.createDataFrame(data=dados,schema=schema)
df_py.printSchema()
df_py.show(truncate=False)

root
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- altura: double (nullable = true)
 |-- peso: double (nullable = true)
 |-- sexo: string (nullable = true)

+-------+-----+------+----+----+
|nome   |idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Douglas|45   |1.85  |70.0|M   |
|Daniela|7    |1.23  |22.0|F   |
|Pedro  |65   |1.75  |87.0|M   |
|Maria  |64   |1.67  |64.0|F   |
|Eduardo|37   |1.82  |96.0|M   |
|Ester  |37   |1.73  |68.0|F   |
+-------+-----+------+----+----+



In [ ]:
# Inserindo uma nova coluna no PySpark via df pandas atualizado
df_py = spark.createDataFrame(df_pd)
df_py.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Douglas|   45|  1.85|  70|   M|
|Daniela|    7|  1.23|  22|   F|
|  Pedro|   65|  1.75|  87|   M|
|  Maria|   64|  1.67|  64|   F|
|Eduardo|   37|  1.82|  96|   M|
|  Ester|   37|  1.73|  68|   F|
+-------+-----+------+----+----+



mudando os nomes das colunas peso e sexo para respectivamente massa e genero
no dataframe **pandas**

In [ ]:
df_pd.rename(columns={'peso':'massa','sexo':'genero'},inplace=True)
display(df_pd)

,nome,idade,altura,massa,genero
0,Douglas,45,1.85,70,M
1,Daniela,7,1.23,22,F
2,Pedro,65,1.75,87,M
3,Maria,64,1.67,64,F
4,Eduardo,37,1.82,96,M
5,Ester,37,1.73,68,F


In [ ]:
# mudando os nomes das colunas peso e sexo para respectivamente massa e genero
# no dataframe PySpark
df_py = df_py.withColumnRenamed("peso","massa").withColumnRenamed("sexo","genero")
df_py.show()

+-------+-----+------+-----+------+
|   nome|idade|altura|massa|genero|
+-------+-----+------+-----+------+
|Douglas|   45|  1.85|   70|     M|
|Daniela|    7|  1.23|   22|     F|
|  Pedro|   65|  1.75|   87|     M|
|  Maria|   64|  1.67|   64|     F|
|Eduardo|   37|  1.82|   96|     M|
|  Ester|   37|  1.73|   68|     F|
+-------+-----+------+-----+------+



In [ ]:
#comando para descobrir a última linha de um df pandas
df_pd.tail(1)

,nome,idade,altura,massa,genero
5,Ester,37,1.73,68,F


Inserindo uma nova linha em uma dataframe pandas

In [ ]:
df_pd.loc[6] = ['Tobias',18,1.67,65,'M']
print(df_pd)
'''
nomedodf.loc[índice de uma nova linha] = [lista com n valores, onde n igual ao número de colunas do df]
'''

      nome  idade  altura  massa genero
0  Douglas     45    1.85     70      M
1  Daniela      7    1.23     22      F
2    Pedro     65    1.75     87      M
3    Maria     64    1.67     64      F
4  Eduardo     37    1.82     96      M
5    Ester     37    1.73     68      F
6   Tobias     18    1.67     65      M


'\nnomedodf.loc[índice de uma nova linha] = [lista com n valores, onde n igual ao número de colunas do df]\n'

In [ ]:
'''
Agora vamos excluir uma linha específica do df
'''
df_pd = df_pd.drop(6)
display(df_pd)

,nome,idade,altura,massa,genero
0,Douglas,45,1.85,70,M
1,Daniela,7,1.23,22,F
2,Pedro,65,1.75,87,M
3,Maria,64,1.67,64,F
4,Eduardo,37,1.82,96,M
5,Ester,37,1.73,68,F


In [ ]:
# inserindo uma nova linha no PySpark
nova_linha = spark.createDataFrame([('Tobias',18,1.67,65.,'M')])
df_py = df_py.union(nova_linha)
df_py.show()

+-------+-----+------+-----+------+
|   nome|idade|altura|massa|genero|
+-------+-----+------+-----+------+
|Douglas|   45|  1.85| 70.0|     M|
|Daniela|    7|  1.23| 22.0|     F|
|  Pedro|   65|  1.75| 87.0|     M|
|  Maria|   64|  1.67| 64.0|     F|
|Eduardo|   37|  1.82| 96.0|     M|
|  Ester|   37|  1.73| 68.0|     F|
| Tobias|   18|  1.67| 65.0|     M|
+-------+-----+------+-----+------+



### Medidas descritivas

In [ ]:
# Medidas descritivas no Pandas
df_pd.describe()

,idade,altura,massa
count,6.000000,6.00000,6.000000
mean,42.500000,1.67500,67.833333
std,21.426619,0.22731,25.615750
min,7.000000,1.23000,22.000000
25%,37.000000,1.68500,65.000000
50%,41.000000,1.74000,69.000000
75%,59.250000,1.80250,82.750000
max,65.000000,1.85000,96.000000


In [ ]:
# Pyspark
df_py.describe().show()

+-------+-------+------------------+-------------------+------------------+------+
|summary|   nome|             idade|             altura|             massa|genero|
+-------+-------+------------------+-------------------+------------------+------+
|  count|      7|                 7|                  7|                 7|     7|
|   mean|   null|              39.0| 1.6742857142857144| 67.42857142857143|  null|
| stddev|   null|21.641010450839243|0.20751362548494195|23.408382706893292|  null|
|    min|Daniela|                 7|               1.23|              22.0|     F|
|    max| Tobias|                65|               1.85|              96.0|     M|
+-------+-------+------------------+-------------------+------------------+------+



* Mediana


* Q1/Q3

* AT
* AIQ
* LS
* LI

## Exercícios

Requisito 1:
Em ambiente PySpark crie uma tabela com quatro atributos sendo o
primeiro qualitativo e os demais como quantitivos e dez registros
(linhas para a tabela) sem valores nulos.

In [ ]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
from pyspark.sql.functions import *
schema = StructType([ \
    StructField('produto', StringType(),True), \
    StructField('preço', IntegerType(),True), \
    StructField('qtd', IntegerType(),True), \
    StructField('peso', DoubleType(), True), \
  ])
dados = [('café',18,2,0.5),
    ('arroz',7,2,0.5),
    ('feijão',8,1,0.5),
    ('macarrão',8,3,0.25),
    ('acúcar',3,1,1.),
    ('sal',2,1,0.5),
    ('ovo',18,20,0.),
    ('pão',8,2,0.48),
    ('suco',14,2,0.75),
    ('leite',4,2,1.)
  ]

df_ex = spark.createDataFrame(data=dados,schema=schema)
df_ex.printSchema()
df_ex.show()

root
 |-- produto: string (nullable = true)
 |-- preço: integer (nullable = true)
 |-- qtd: integer (nullable = true)
 |-- peso: double (nullable = true)

+--------+-----+---+----+
| produto|preço|qtd|peso|
+--------+-----+---+----+
|    café|   18|  2| 0.5|
|   arroz|    7|  2| 0.5|
|  feijão|    8|  1| 0.5|
|macarrão|    8|  3|0.25|
|  acúcar|    3|  1| 1.0|
|     sal|    2|  1| 0.5|
|     ovo|   18| 20| 0.0|
|     pão|    8|  2|0.48|
|    suco|   14|  2|0.75|
|   leite|    4|  2| 1.0|
+--------+-----+---+----+



In [ ]:
df_ex.summary()

summary,produto,preço,qtd,peso
count,10,10,10,10
mean,null,9.0,3.6,0.548
stddev,null,5.811865258054231,5.796550698475775,0.30774448275585164
min,acúcar,2,1,0.0
25%,null,4,1,0.48
50%,null,8,2,0.5
75%,null,14,2,0.75
max,suco,18,20,1.0


Requisito 2:
Em ambiente PySpark gerar um describe da tabela gerada no requisito anterior e recebê-lo com um df pyspark.

In [ ]:
medidas=df_ex.describe()
medidas.show()

+-------+-------+-----------------+-----------------+-------------------+
|summary|produto|            preço|              qtd|               peso|
+-------+-------+-----------------+-----------------+-------------------+
|  count|     10|               10|               10|                 10|
|   mean|   null|              9.0|              3.6|              0.548|
| stddev|   null|5.811865258054231|5.796550698475775|0.30774448275585164|
|    min| acúcar|                2|                1|                0.0|
|    max|   suco|               18|               20|                1.0|
+-------+-------+-----------------+-----------------+-------------------+



Requisito 3: Em ambiente PySpark

Selecionar a linha de desvio padrão do df gerar uma lista no ambiente python e armazená-lo com o nome sd.

In [ ]:
sd=medidas.drop("produto","summary").filter(medidas["summary"]=="stddev")
std=list(sd.collect()[0])
display(std)

['5.811865258054231', '5.796550698475775', '0.30774448275585164']

In [ ]:
type(std)

list

Requisito 4:
Em ambiente Python, manipular essa lista e a partir dela gerar uma lista
com todos os valores numéricos da lista sd elevado ao quadrado e receber essa nova lista com o nome var

In [ ]:
var=[]
sd=[]
for i in std:
  sd.append(float(i))
  var.append(float(i)**2)
print(sd)
print(var)

[5.811865258054231, 5.796550698475775, 0.30774448275585164]
[33.77777777777778, 33.599999999999994, 0.09470666666666668]


Requisito 5:
Transformar cada lista do requisito anterior em um df py spark

In [ ]:
sd.insert(0,'desvio padrao')
var.insert(0,'variancia')
sd=tuple(sd)
var=tuple(var)

In [ ]:
sd_py1 = spark.createDataFrame([sd])
sd_py1.printSchema()
sd_py1.show()

var_py1 = spark.createDataFrame([var])
var_py1.printSchema()
var_py1.show()

root
 |-- _1: string (nullable = true)
 |-- _2: double (nullable = true)
 |-- _3: double (nullable = true)
 |-- _4: double (nullable = true)

+-------------+-----------------+-----------------+-------------------+
|           _1|               _2|               _3|                 _4|
+-------------+-----------------+-----------------+-------------------+
|desvio padrao|5.811865258054231|5.796550698475775|0.30774448275585164|
+-------------+-----------------+-----------------+-------------------+

root
 |-- _1: string (nullable = true)
 |-- _2: double (nullable = true)
 |-- _3: double (nullable = true)
 |-- _4: double (nullable = true)

+---------+-----------------+------------------+-------------------+
|       _1|               _2|                _3|                 _4|
+---------+-----------------+------------------+-------------------+
|variancia|33.77777777777778|33.599999999999994|0.09470666666666668|
+---------+-----------------+------------------+-------------------+



Requisito 6:
Em ambiente Pyspark juntar os dois dfs criados no requisito anterior e
armazená-lo em um df pyspark chamado sd_var

In [ ]:
sd_var = sd_py1.union(var_py1)
sd_var.show()

+-------------+-----------------+------------------+-------------------+
|           _1|               _2|                _3|                 _4|
+-------------+-----------------+------------------+-------------------+
|desvio padrao|5.811865258054231| 5.796550698475775|0.30774448275585164|
|    variancia|33.77777777777778|33.599999999999994|0.09470666666666668|
+-------------+-----------------+------------------+-------------------+



Requisito 7:
Em ambiente Pyspark gerar uma junção do df da variância com o df das medidas

In [ ]:
medidas=medidas.drop("produto").union(var_py1)
medidas.show()

+---------+-----------------+------------------+-------------------+
|  summary|            preço|               qtd|               peso|
+---------+-----------------+------------------+-------------------+
|    count|               10|                10|                 10|
|     mean|              9.0|               3.6|              0.548|
|   stddev|5.811865258054231| 5.796550698475775|0.30774448275585164|
|      min|                2|                 1|                0.0|
|      max|               18|                20|                1.0|
|variancia|33.77777777777778|33.599999999999994|0.09470666666666668|
+---------+-----------------+------------------+-------------------+



Requisito 8:
criar uma lista que contenha as modas de todos os atributos quantitativos do df inicial e inseri-la no df pyspark de medidas

In [ ]:
moda=[df_ex.groupby(i).count().orderBy("count", ascending=False).first()[0] for i in df_ex.columns]
moda.pop(0)

'leite'

In [ ]:
df_ex.summary()

summary,produto,preço,qtd,peso
count,10,10,10,10
mean,null,9.0,3.6,0.548
stddev,null,5.811865258054231,5.796550698475775,0.30774448275585164
min,acúcar,2,1,0.0
25%,null,4,1,0.48
50%,null,8,2,0.5
75%,null,14,2,0.75
max,suco,18,20,1.0


Requisito 9:
Em ambiente Pyspark e Python e de forma semelhante ao que foi feito nos
requisitos anteriores

Calcule e adicione ao df de medidas do pyspark as seguintes linhas com seus
respectivos valores para as medidas:
* Mediana
* Q1
* Q3

In [ ]:
quartis=df_ex.approxQuantile(['preço','qtd','peso'],[0.25,0.5,0.75],0)
print(quartis)

[[4.0, 8.0, 14.0], [1.0, 2.0, 2.0], [0.48, 0.5, 0.75]]


In [ ]:
q1=[quartis[0][0],quartis[1][0],quartis[2][0]]
med=[quartis[0][1],quartis[1][1],quartis[2][1]]
q3=[quartis[0][2],quartis[1][2],quartis[2][2]]

print(q1)
print(med)
print(q3)

[4.0, 1.0, 0.48]
[8.0, 2.0, 0.5]
[14.0, 2.0, 0.75]


* Min/Max

In [ ]:
min=medidas.collect()[3]
min

Row(summary='min', preço='2', qtd='1', peso='0.0')

In [ ]:
float(min['peso'])

mínimo=[float(min['preço']),float(min['qtd']),float(min['peso'])]
mínimo

[2.0, 1.0, 0.0]

In [ ]:
min = list(medidas.collect()[3])
min.pop(0)
minimo= [float(i) for i in min]

max = list(medidas.collect()[4])
max.pop(0)
maximo = [float(i) for i in max]

print(minimo)
print(maximo)

[2.0, 1.0, 0.0]
[18.0, 20.0, 1.0]


* AT
* AIQ
* LS
* LI

In [ ]:
at = []
aiq = []
li = []
ls = []

for i in range(len(minimo)):
  at.append(maximo[i]-minimo[i])
  aiq.append(q3[i]-q1[i])
  li.append(q1[i]-1.5*aiq[i])
  ls.append(q3[i]+1.5*aiq[i])

q1.insert(0, 'Q1')
med.insert(0, 'mediana')
q3.insert(0, 'Q3')
minimo.insert(0,'min')
maximo.insert(0,'max')
at.insert(0, 'AT')
aiq.insert(0, 'AIQ')
li.insert(0, 'LI')
ls.insert(0,'LS')

print(q1)
print(med)
print(q3)
print(minimo)
print(maximo)
print(at)
print(aiq)
print(li)
print(ls)

['Q1', 4.0, 1.0, 0.48]
['mediana', 8.0, 2.0, 0.5]
['Q3', 14.0, 2.0, 0.75]
['min', 2.0, 1.0, 0.0]
['max', 18.0, 20.0, 1.0]
['AT', 16.0, 19.0, 1.0]
['AIQ', 10.0, 1.0, 0.27]
['LI', -11.0, -0.5, 0.07499999999999996]
['LS', 29.0, 3.5, 1.155]


In [ ]:
adc = spark.createDataFrame([q1,med,q3,at,aiq,li,ls])
adc.show()

medidas=medidas.union(adc)
medidas.show()

+-------+-----+----+-------------------+
|     _1|   _2|  _3|                 _4|
+-------+-----+----+-------------------+
|     Q1|  4.0| 1.0|               0.48|
|mediana|  8.0| 2.0|                0.5|
|     Q3| 14.0| 2.0|               0.75|
|     AT| 16.0|19.0|                1.0|
|    AIQ| 10.0| 1.0|               0.27|
|     LI|-11.0|-0.5|0.07499999999999996|
|     LS| 29.0| 3.5|              1.155|
+-------+-----+----+-------------------+

+---------+-----------------+------------------+-------------------+
|  summary|            preço|               qtd|               peso|
+---------+-----------------+------------------+-------------------+
|    count|               10|                10|                 10|
|     mean|              9.0|               3.6|              0.548|
|   stddev|5.811865258054231| 5.796550698475775|0.30774448275585164|
|      min|                2|                 1|                0.0|
|      max|               18|                20|                1

### Automação Quartis

In [ ]:
from pyspark.sql.functions import col
#Lista todas as Colunas do dataframe
todas_colunas = [coluna for coluna, tipo in df_ex.dtypes ]
#Lista todas as Colunas do dataframe
colunas_numericas = [coluna for coluna, tipo in df_ex.dtypes if tipo in ['double', 'float', 'decimal', 'int', 'long']]
# Calculando os quartis de todos os atributos quantitativos
quartis = df_ex.approxQuantile(colunas_numericas, [0.25,0.5,0.75], 0)
print(colunas_numericas,quartis)

['preço', 'qtd', 'peso'] [[4.0, 8.0, 14.0], [1.0, 2.0, 2.0], [0.48, 0.5, 0.75]]


### Verificações Extras

In [ ]:
from pyspark.sql.functions import col, count, when, isnan, isnull

# Número de linhas
num_linhas = df_py.count()
print("Número de linhas: ", num_linhas)

# Número de colunas
num_colunas = len(df_py.columns)
print("Número de colunas: ", num_colunas)

# Nomes das colunas e seus tipos
print("Esquema do DataFrame:")
df_py.printSchema()

# Contagem de valores não nulos por coluna
print("Contagem de valores não nulos por coluna:")
df_py.agg(*[(count(when(col(c).isNotNull(), c)).alias(c)) for c in df_py.columns]).show()

# Contagem de valores nulos por coluna
print("Contagem de valores nulos por coluna:")
df_py.agg(*[(count(when(isnull(col(c)), c)).alias(c)) for c in df_py.columns]).show()

# Contagem de valores ausentes por coluna
print("Contagem de valores ausentes por coluna:")
df_py.agg(*[(count(when(isnan(col(c)) | isnull(col(c)), c)).alias(c)) for c in df_py.columns]).show()